In [36]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime
try:
  import cplex
except:
  !pip install cplex
  import cplex
import math

#get data
tickers = ['AAPL', 'NVDA', 'TSLA', 'MSFT', 'AMZN', 'GLD']
start_date = '2025-03-27'
end_date = datetime.today().strftime('%Y-%m-%d')

df = pd.DataFrame()
for t in tickers:
  df[t]=yf.download(t,start=start_date,end=end_date)['Close']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
#calculate return covariance and mean
df_return = df.pct_change().dropna()
mu, Q = df_return.mean(), df_return.cov()
print(mu)
print(Q)

AAPL   -0.002134
NVDA    0.001387
TSLA    0.004491
MSFT    0.000614
AMZN   -0.002239
GLD     0.004052
dtype: float64
          AAPL      NVDA      TSLA      MSFT      AMZN       GLD
AAPL  0.002650  0.002650  0.003263  0.001426  0.002010  0.000291
NVDA  0.002650  0.003470  0.003917  0.001736  0.002430  0.000128
TSLA  0.003263  0.003917  0.005292  0.002087  0.002823  0.000158
MSFT  0.001426  0.001736  0.002087  0.000982  0.001261  0.000096
AMZN  0.002010  0.002430  0.002823  0.001261  0.001976  0.000056
GLD   0.000291  0.000128  0.000158  0.000096  0.000056  0.000345


In [38]:
def strat_max_Sharpe(mu, Q, r_rf):
    cpx = cplex.Cplex()
    cpx.objective.set_sense(cpx.objective.sense.minimize)

    # Number of assets
    n = len(mu)

    # 1) Linear objective coefficients => 0 for all variables (only a quadratic term).
    c = [0.0]*(n + 1)

    # 2) Variable bounds lb => y_i >= 0, kappa >= 1e-6
    # ub => y_i and kappa can be any number to infinity
    lb = [0.0]*n + [1e-6]
    ub = [cplex.infinity]*(n+1)

    # --------------------------------------------------------------
    # 3) Two constraints:
    #   (a) sum_i((mu_i - r_rf/252) * y_i) = 1
    #   (b) sum_i(y_i) - kappa = 0
    # --------------------------------------------------------------
    Atilde = [[[0,1], [mu[i]-r_rf / 252.0, 1.0]] for i in range(n)]

    # The (n+1)-th variable is kappa, so row 0 -> 0.0, row 1 -> -1.0
    Atilde.append([[0, 1], [0.0, -1.0]])

    my_sense = "EE"         # two equality constraints
    rhs_vals = [1.0, 0.0]   # right-hand sides

    cpx.linear_constraints.add(rhs=rhs_vals, senses=my_sense)

    var_names = [f"y_{i}" for i in range(n)] + ["kappa"]
    cpx.variables.add(obj=c, lb=lb, ub=ub, columns=Atilde, names=var_names)

    # --------------------------------------------------------------
    # 4) Quadratic objective: y^T Q y
    # --------------------------------------------------------------
#     Qmat = []
#     for i in range(n):
#         Qmat.append([list(range(n)), Q[i]])  # row for y_i
    Qmat =  [[list(range(n)), Q[i]] for i in range(n)]

    # For kappa (the last var), we have zero row
    Qmat.append([[n], [0.0]])

    cpx.objective.set_quadratic(Qmat)

    # --------------------------------------------------------------
    # 5) Solve the model
    # --------------------------------------------------------------
    alg = cpx.parameters.lpmethod.values
    cpx.parameters.qpmethod.set(alg.concurrent)
    #cpx.set_results_stream(None)

    cpx.solve()

    # --------------------------------------------------------------
    # 6) Retrieve the solution
    # --------------------------------------------------------------
    numcols = cpx.variables.get_num()  # should be n+1
    x_opt = [cpx.solution.get_values(j) for j in range(numcols)]

    y_opt = x_opt[:n]
    kappa_opt = x_opt[n]

    # Portfolio weights: w_i = y_i / kappa
    w_ms = np.array([y_i / kappa_opt for y_i in y_opt])

    # Calculate y^T Q y
    #  => We'll do a straightforward summation
    yQy = 0.0
    for i in range(n):
        for j in range(n):
            yQy += y_opt[i] * Q[i][j] * y_opt[j]

    # Maximum Sharpe ratio = 1 / sqrt(y^T Q y)
    max_sharpe = 1.0 / math.sqrt(yQy)

    return w_ms, max_sharpe

In [39]:
strat_max_Sharpe(np.array(mu), np.array(Q), 0.0429/252)

Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_QPMethod                                6
Parallel mode: deterministic, using up to 2 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 1 thread...
Number of nonzeros in lower triangle of Q = 15
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 6
  Integer space required    = 6
  Total non-zeros in factor = 21
  Total FP ops to factor    = 91
Tried aggregator 1 time.
No QP presolve or aggregator reductions.
Presolve time = 0.02 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1    Objective     =             0.000000

Dual simplex solved model.



(array([0.        , 0.        , 0.04198558, 0.        , 0.        ,
        0.95801442]),
 0.2212659731785513)